# Tutorial: Como adquirir dados em um ambiente de testes crypto

#### Autor: Raul Ikeda (rauligs@insper.edu.br)
#### Data: Ago/2025
#### Versão: 1.0.0

Esse documento é um tutorial de um protótipo de como consumir dados em um ambiente de testes (sandbox) em uma exchange crypto. Não é escopo desse material o estudo ou desenvolvimento de modelos ou recomendação de alocação. É exigido um pouco de conhecimento de programação em Python e os principais conceitos de microestrutura de mercado.

**IMPORTANTE**: Esse projeto é completamente experimental e instável. Ele não deve ir para produção e utilizar uma conta real. O autor não se responsabiliza por eventuais perdas e bugs no código.

---

# I - Configurações iniciais

O objetivo dessa etapa é criarmos e configurarmos uma conta na TESTNET da Binance. 

A testnet, normalmente é chamada de `Sandbox`, um lugar onde podemos rodar provas de conceito sem afetar o mundo real. Serve como um ambiente de testes que funciona análogo ao ambiente de produção, mas sem consequências.

### 1- O primeiro passo é criar uma conta no GitHub (https://github.com/)

GitHub é um serviço de armazenamento de arquivos no protocolo Git. Ele faz versionamento automático e rastreamento de arquivos e cuida da colaboração de times. Ele funciona muito bem para códigos de programação, mas pode ser usados para outros fins. É como a ferramenta de rastreio do Office, mas com muito mais funcionalidades.

O GitHub possui funcionalidades extras como actions (CI/CD), issues, project, milestones, kanban, etc, o que permite fazer o gerenciamento completo de um projeto. Outros concorrentes que usam o protocolo git: Gitlab, bitbucket, etc.

### 2- Realizar o login com a conta do GitHub no site: https://testnet.binance.vision/

Deve aparecer: autenticado como *test user*.

### 3- Gerar as chaves para usar a API (API Keys)

Há duas opções:
- Generate HMAC-SHA-256 Key
- Register Public Key

Se você não possui chave pública, seguir pela primeira opção. Adicionar uma descrição e o sistema devolvera dois valores: **API Key** e **Secret Key**. Não feche a página ainda!

### 4- Criar um arquivo novo chamado token.toml e adicionar o seguinte conteúdo (LEIA! NÃO COPIAR E COLAR):

Esse arquivo vai conter os dados de autenticação para Binance:

```toml
[keys]
api_key = 'Colocar aqui o API Key da página'
api_secret = 'Colocar aqui o Secret Key da página'
```

Importante: Leia o conteúdo do arquivo antes de copiar e colar. As aspas são importantes.

Um arquivo toml (leia-se *Tômel*) é um arquivo de configuração que será lido pelo código principal. O formato toml é interessante porque ele é amigável para a leitura de um ser humano e permite que separemos os dados por *sections*. Existem algumas alternativas ao toml: yaml, ini, plain text, etc. Ref: https://github.com/toml-lang/toml

Uma boa prática é não colocar chaves/senhas no código de programação. Assim você corre menos risco de vazamento de chaves, desde que tome cuidado para não vazar o arquivo toml - se você está trabalhando com um repositório no GitHub, não esquecer de colocar no .gitignore. Uma outra alternativa é colocar como variável de ambiente (Windows ou Linux), isso gera uma camada extra de segurança, mas vai exigir uma instalação do produto no usuário. https://medium.com/twodigits/keep-passwords-out-of-source-code-why-and-how-e84f9004815a

### 5- Abrir o arquivo toml

Vamos agora abrir em python o arquivo e imprimir o valor das chaves. Note claramente que o código não possui as chaves escritas.

Instalar a biblioteca toml: 

```shell
pip install toml
```

E abrir o arquivo no python

```python
import toml
 
with open('token.toml', 'r') as f:
    config = toml.load(f)
    
api_key = config['keys']['api_key']
api_secret = config['keys']['api_secret']
```

Imprimir o conteúdo das variáveis *api_key* e *secret_key* e conferir com o exibido na página.

---

Uma vez de posse das chaves (keys), estamos prontos para começar a interagir com a Exchange diretamente. Essse modelo de autenticação substitui o login com senha e fatores de autenticação, permitindo interagir diretamente por API, sem a necessidade de ação manual humana.

API (Application Programming Interface) é uma forma que os sistemas fornecem para que os usários consigam programar requisições que atuarão no sistema, mas de forma controlada. Permite a automação de processos. Um exemplo comum de API é realizar mudanças dentro de uma planilha Excel via VBA. Ler mais em: https://aws.amazon.com/pt/what-is/api/

Vale mencionar novamente: as chaves são perigosíssimas e vazá-las em um ambiente real é como perder o cartão e a senha, sob o risco de perder todos os ativos e fundos. Por isso a necessidade de um tratamento especial. Em um ambiente corporativo, as chaves são tratadas com sigilo absoluto, utilizando um sistema de **vault multisig**, ambiente de produção com acesso restrito ou chaves parcialmente compartilhadas (ninguém possui a chave toda, apenas pedaços). Ler mais em: https://www.akeyless.io/secrets-management-glossary/vault-as-a-service/


In [1]:
# Escreva o código para abrir o arquivo TOML e imprimir as chaves para conferência

# II - Verificando o Saldo atual

Após ter as chaves para o ambiente de testes, o primeiro passo será olharmos o saldo atual da carteira.

É importante sempre no início de algum programa fazer o batimento das posições, pois pode ter acontecido enquanto o sistema não estava no ar, seja alguma ação manual ou algo assíncrono. Lembrem-se que não controlamos quando os eventos acontecem.

A API disponibilizada pela Binance permitiria realizar uma consulta diretamente no navegador, porém formatar essa chamada não é nada prático. Podemos também realizar via python usando a biblioteca requests.

Mas ainda não é muito prático. Existe ainda uma terceira alternativa: utilizar uma biblioteca que encapsula as chamadas de API por funções, tornando a vida de usuário da plataforma significantemente mais fácil. Essas bibliotecas são chamadas de *wrappers*, elas não contém regras de negócio e simplesmente automatizam tarefas que realizam uma ação que consumiria muitas linhas de código.

Vamos usar a biblioteca python-binance. É fácil notar que ela só vai funcionar para essa Exchange. Caso procure uma solução mais genérica, de certa forma vai ter que abrir mão do wrapper.

```python
!pip install python-binance
```

Uma vez instalada, precisamos começar a olhar a documentação da biblioteca/API: https://python-binance.readthedocs.io/en/latest/

Começando com um código bem simples:
```python
from binance.client import Client
client = Client(api_key, api_secret, testnet=True)
position = client.get_account()
print(position)
```

Note que o resultado vai ser um dicionário enorme. E o que interessa agora é o valor da chave *balances*

```python
print(position['balances'])
```

O resultado agora é uma lista de dicionários com 3 chaves:
- asset: nome do ativo
- free: quantidade disponível na carteira
- locked: quantidade travada na carteira. Esse valor é importante pois não será possível transacionar a quantidade que estão travadas - por exemplo: estão prometidas com uma ordem de venda limitada que não foi executada.

A posição total atual é a soma de quantidade *free* e da quantidade *locked*.

Exercício: Você consegue imprimir sua posição total atual para cada um dos ativos?

**ATENÇÃO**: não, você não está rico. Essa posição é fictícia e não vale como um ativo real. Ela só existe no ambiente de testes.

Tudo que estamos fazendo até aqui está no grande grupo chamado SPOT - uma definição para ativos à vista (Assim como ações na bolsa). A Binance ainda conta com os derivativos: FUTURES e OPTIONS. Uma consequência de ser SPOT é não haver a influência da curva de juros e não poder operar short diretamente.

---

A variável client tem um papel fundamental: ela centraliza a chamada das funções usando os dados da conta (chaves). Tome um tempo lendo a documentação:

- https://python-binance.readthedocs.io/en/latest/overview.html
- https://python-binance.readthedocs.io/en/latest/general.html
- https://python-binance.readthedocs.io/en/latest/binance.html#binance.client.Client - aqui vocês podem uma boa ideia do que pode ser feito com o objeto client, ou seja, todas as funcionalidades embutidas via function.

Também chegou a hora de ler sobre F.A.Q (Frequently Answered Questions) na Binance:

- https://testnet.binance.vision/

Por exemplo, é possível tirar um *snapshot* dos preços atuais:

```python
prices = client.get_all_tickers()
print(prices)
```

Ou a foto do book atual (LOB)

```python
lob = client.get_order_book(symbol='BTCUSDT')
print(lob)
```

Exercício: Você consegue calcular qual o seu patrimônio atual em USDT? Não esqueça das quantidades **locked**

---

Um snapshot é uma foto atual do mercado. É uma forma síncrona (request/response) de visualizar os preços. Porém o mercado é completamente assíncrono e os preços são atualizados em tempos irregulares, conforme as ordens vão preenchendo. É inviável ficar realizando requests a todo momento e o ideal era ter uma forma de ser avisado automaticamente quando um preço é atualizado. Para isso vamos utilizar um *websocket*.

A API da Binance (do servidor e não da biblioteca) utiliza um protocolo chamado REST (https://en.wikipedia.org/wiki/REST). É um padrão de mercado que utiliza o HTTP (o mesmo de páginas na internet) via request/response de forma síncrona e possui um rígido modelo de utilização. Sistemas modernos costumam seguir esse protocolo, mas há alternativas viáveis e de nicho como: GraphQL, gRPC, websockets, etc.

In [3]:
# Faça o request da posição atual e imprima
import dotenv
import os
import requests
from binance.client import Client

dotenv.load_dotenv('.env')
api_key=os.getenv('API_KEY')
api_secret=os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet=True)
position = client.get_account()
# print(position)
print(position['balances'])


[{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'}, {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '1778.00000000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '228.00000000', 'locked': '0.00000000'}, {'asset': 'NEO', 'free': '92.00000000', 'locked': '0.00000000'}, {'asset': 'QTUM', 'free': '260.00000000', 'locked': '0.00000000'}, {'asset': 'GAS', 'free': '179.00000000', 'locked': '0.00000000'}, {'asset': 'LRC', 'free': '6693.00000000', 'locked': '0.00000000'}, {'asset': 'ZRX', 'free': '2325.00000000', 'locked': '0.00000000'}, {'asset': 'KNC', 'free': '1778.00000000', 'locked': '0.00000000'}, {'asset': 'IOTA', 'free': '3197.00000000', 'locked': '0.00000000'}, {'asset': 'LINK', 'free': '38.00000000', 'locked': '0.00000000'}, {'asset'

In [4]:
# Faça o request dos preços e imprima

prices = client.get_all_tickers()
print(prices)

[{'symbol': 'ETHBTC', 'price': '0.04103000'}, {'symbol': 'LTCBTC', 'price': '0.00102800'}, {'symbol': 'BNBBTC', 'price': '0.00774700'}, {'symbol': 'BTCUSDT', 'price': '110973.77000000'}, {'symbol': 'ETHUSDT', 'price': '4559.79000000'}, {'symbol': 'TRXBTC', 'price': '0.00000319'}, {'symbol': 'XRPBTC', 'price': '0.00002692'}, {'symbol': 'BNBUSDT', 'price': '855.98000000'}, {'symbol': 'LTCUSDT', 'price': '113.89000000'}, {'symbol': 'LTCBNB', 'price': '0.13240000'}, {'symbol': 'XRPUSDT', 'price': '2.97970000'}, {'symbol': 'XRPBNB', 'price': '0.00347140'}, {'symbol': 'TRXBNB', 'price': '0.00041070'}, {'symbol': 'TRXUSDT', 'price': '0.35100000'}, {'symbol': 'NEOBTC', 'price': '0.00006560'}, {'symbol': 'QTUMETH', 'price': '0.00065900'}, {'symbol': 'GASBTC', 'price': '0.00003050'}, {'symbol': 'BNBETH', 'price': '0.18790000'}, {'symbol': 'LRCBTC', 'price': '0.00000079'}, {'symbol': 'LRCETH', 'price': '0.00001877'}, {'symbol': 'QTUMBTC', 'price': '0.00002781'}, {'symbol': 'ZRXBTC', 'price': '0.0

In [ ]:
# Exercício: calcule seu patrimônio atual em USDT
carteira = 0
for 

# III - Recebendo Preços em Tempo Real

Agora que você já está acostumado a ler a documentação. Vamos direto para o ponto principal:

https://python-binance.readthedocs.io/en/latest/websockets.html

Nesse ponto a biblioteca sinaliza que existem 2 formas de receber os preços via stream: ThreadedWebsocketManager ou BinanceSocketManager.

Outro ponto importantíssimo: Existe o mono canal (para 1 stream) e o canal multiplexado (para diversos streams).

Outra confusão que pode surgir: existem diversos formatos de recebimento de preço.

- Depth: fornece o LOB. Precisa especificar a profundidade desejada (ex: 5 best bids/asks)
- Kline: fornece o OHLC. Precisa informar a peridicidade (ex: 15 minutos)
- Book Ticker: Best bid/ask do LOB.
- Etc. Ver: https://binance-docs.github.io/apidocs/spot/en/#websocket-market-streams

Vamos fazer um primeiro código para receber os dados agregados de minuto a minuto (kline) em tempo real:

```python
async def main(api_key, api_secret):
    print('Connection started!')
    client = None
    try:
        client = await AsyncClient.create(api_key=api_key, api_secret=api_secret, testnet=True)
        bm = BinanceSocketManager(client)
        # start any sockets here, i.e a trade socket
        ts = bm.kline_socket('BTCUSDT', interval=KLINE_INTERVAL_1MINUTE)
        # then start receiving messages
        async with ts as tscm:
            while True:
                res = await tscm.recv()
                print(f'Evento: {res}')
                
    except asyncio.CancelledError:
        print('Task was cancelled!')
    finally:
        if client:
            await client.close_connection()
        print('Connection finished!')    

loop = asyncio.get_event_loop()
task = loop.create_task(main(api_key, api_secret))

# Espere 15 segundos para cancelar a tarefa main()
await asyncio.sleep(15)
task.cancel()
```

O código acima foi modificado dos exemplos da documentação para funcionar dentro de um notebook.

Alguns pontos importantes:
- O código roda em uma thread separada, por isso ele não bloqueia a thread principal do notebook (não fica esperando terminar de rodar)
- Como consequência, para parar a transmissão, você precisa rodar: task.cancel(). Ele rodará automaticamente após 15 segundos
- Não rode nenhuma célula enquanto espera

Olhe para a mensagem de saída. Segue um exemplo do que significa cada variável:

```json
{
    "e": "kline",                                   # event type
    "E": 1499404907056,                             # event time
    "s": "ETHBTC",                                  # symbol
    "k": {
        "t": 1499404860000,                 # start time of this bar
        "T": 1499404919999,                 # end time of this bar
        "s": "ETHBTC",                              # symbol
        "i": "1m",                                  # interval
        "f": 77462,                                 # first trade id
        "L": 77465,                                 # last trade id
        "o": "0.10278577",                  # open
        "c": "0.10278645",                  # close
        "h": "0.10278712",                  # high
        "l": "0.10278518",                  # low
        "v": "17.47929838",                 # volume
        "n": 4,                                             # number of trades
        "x": false,                                 # whether this bar is final
        "q": "1.79662878",                  # quote volume
        "V": "2.34879839",                  # volume of active buy
        "Q": "0.24142166",                  # quote volume of active buy
        "B": "13279784.01349473"    # can be ignored
        }
}
```
Um detalhe: ele vai enviar todas as modificações que acontecer no candle e não necessariamente na virada do minuto. É preciso selecionar o evento quando há a virada do minuto, ou seja, quando 'x' for True.

Adicionar o processamento no começo da célula:

```python
import asyncio

from binance import AsyncClient, BinanceSocketManager
from binance.enums import *
from datetime import datetime

prices = []

async def process_kline(message):
    if message['k']['x'] or True:
        price = [datetime.utcfromtimestamp(message['k']['t']/1000), # Timestamp
                 float(message['k']['o']), # Open
                 float(message['k']['h']), # High
                 float(message['k']['l']), # Low
                 float(message['k']['c']), # Close
                 float(message['k']['v']), # Volume
                ]
        prices.append(price)
        print(price)

async def main(api_key, api_secret):
    ...
    # Resto do código
        
```

Observações importantes:
- Como o timestamp vem com os milissegundos, é preciso dividr por mil
- Os preços vem em string, portanto precisa transformar em float
- O uso do async permite que a função rode como co-rotina

Agora adicione antes do print do evento:
```python
                ...
                res = await tscm.recv()
                
                # Adicione essa linha:
                await process_kline(res)
            
                print(f'Evento: {res}')
                ...

```

Rode novamente e compare a saída dos dados processados com os não processados.

---

Para finalizar:

- Remova os prints do código
- Remova o sleep e o cancel do final do código
- Reserve!

O *asyncio* (https://docs.python.org/3/library/asyncio.html) é uma biblioteca em Python projetada para escrever código concorrente usando a sintaxe *async*/*await*. Utilizando *async*, você define funções que são corotinas, as quais podem ser interrompidas durante sua execução para que outras operações sejam executadas, melhorando a eficiência, especialmente em tarefas que envolvem espera, como o websocket. Já *await* é usada para pausar a execução da corotina até que uma operação  seja concluída, permitindo que outras tarefas sejam executadas durante essa espera. Esse modelo não apenas simplifica o código assíncrono, tornando-o similar ao síncrono, mas também facilita o manejo de conexões de rede e outras operações que podem bloquear a execução do código. Por se tratar de um websocket, precisamos da biblioteca ou a conexão iria bloquear todo o processamento do código, não permitindo realizar outras operações enquanto não chega nenhum preço.


In [ ]:
import asyncio

from binance import AsyncClient, BinanceSocketManager
from binance.enums import *
from datetime import datetime

prices = []

async def process_kline(message):
    if message['k']['x'] or True:
        price = [datetime.utcfromtimestamp(message['k']['t']/1000), # Timestamp
                 float(message['k']['o']), # Open
                 float(message['k']['h']), # High
                 float(message['k']['l']), # Low
                 float(message['k']['c']), # Close
                 float(message['k']['v']), # Volume
                ]
        prices.append(price)
        print(price)

        # Coloque aqui a sua estratégia
        

async def main(api_key, api_secret):
    print('Connection started!')
    client = None
    try:
        client = await AsyncClient.create(api_key=api_key, api_secret=api_secret, testnet=True)
        bm = BinanceSocketManager(client)
        # start any sockets here, i.e a trade socket
        ts = bm.kline_socket('BTCUSDT', interval=KLINE_INTERVAL_1MINUTE)
        # then start receiving messages
        async with ts as tscm:
            while True:
                res = await tscm.recv()
                await process_kline(res)
                
    except asyncio.CancelledError:
        print('Task was cancelled!')
    finally:
        if client:
            await client.close_connection()
        print('Connection finished!')    

loop = asyncio.get_event_loop()
task = loop.create_task(main(api_key, api_secret))

# Para após 15 segundos, e cancela a tarefa
await asyncio.sleep(15)
task.cancel()

# IV - Enviando Ordens

Como já visto no curso, os modelos irão enviar sinais para a execução de ordens. O ato de envio é síncrono, ou seja, você consegue prever quando uma ordem será enviada, porém a execução da ordem não. O preenchimento da ordem acontece em um momento posterior ao envio e pode demorar um tempo variável a depender das condições do mercado.

Portanto sabemos que o envio de ordens pode ser via client, mas o recebimento dos preenchimentos deve ser via websocket, assim como os preços no passo anterior.

Primeiro o envio: 

```python

# client foi definido na etapa I

client.create_order(symbol="BTCUSDT",             # Pair to trade
                    side=SIDE_SELL,               # Order side
                    type=ORDER_TYPE_LIMIT,        # Order type (market or limit)
                    quantity=0.1,                 # Quantity (you must have position to sell)
                    price = 90000,                # Price (try to avoid filling the order)
                    timeInForce=TIME_IN_FORCE_GTC # How long the market you last (GTC = good til cancel)
                   )
```

Retorno:

```python
    {'symbol': 'BTCUSDT',
     'orderId': 1856309,                          # Order Id
     'orderListId': -1,
     'clientOrderId': 'KLDf4Yj87IocINTFp5crCG',   # Client Id
     'transactTime': 1713298991524,               # Timestamp
     'price': '90000.00000000',
     'origQty': '0.10000000',
     'executedQty': '0.00000000',                 # Executed quantity (initially zero)
     'cummulativeQuoteQty': '0.00000000',
     'status': 'NEW',                             # Order Status (NEW order)
     'timeInForce': 'GTC',
     'type': 'LIMIT',
     'side': 'SELL',
     'workingTime': 1713298991524,
     'fills': [],                                 # List of fills (should be empty)
     'selfTradePreventionMode': 'EXPIRE_MAKER'}
```

Como visto no curso, essa ordem irá compor o LOB, especificamente no ASK.

O problema agora é que por se tratar de uma função síncrona, não iremos receber nenhuma atualização da ordem. Inclusive, se você nunca mais mexer nela, ficará pendurada para até ser preenchida (ou cancelada pela Exchange). No caso específico da Testnet, a Exchange limpa os dados de tempos em tempos (ver o FAQ). Mas em produção isso poderia se tornar um pesadelo.

Antes de começarmos a capturar os eventos de ordem, vamos primeiro ver outras formas de parametrizar uma ordem e também como listar e cancelar ordens de forma síncrona. 

Você já deve estar ambientado com a documentação e deve conseguir achar o pedaço referente ao envio:

1. Especificamente, a função create_order pertence a um objeto Client.
2. Em https://python-binance.readthedocs.io/en/latest/binance.html#module-binance.client
3. Se olharmos a forma como criamos o objeto: Utilizamos a classe Client do módulo binance.client
4. Portanto procuramos a definição: class binance.client.Client(...)
5. Logo abaixo temos os métodos (funções) da classe. especificamente queremos: create_order(...)
6. Cuidado ao usar CTRL+F, tem vários métodos parecidos com esse (não há margem para erros)

Ao achar, vocês devem ver:

- Descritivo da função
- URL de referência da API da Binance (IMPORTANTE!)

Em seguida os parâmetros:

```txt
Parameters:

-symbol (str) – required
-side (str) – required
-type (str) – required
-timeInForce (str) – required if limit order
-quantity (decimal) – required
-quoteOrderQty (decimal) – amount the user wants to spend (when buying) or receive (when selling) of the 
quote asset, applicable to MARKET orders
-price (str) – required
-newClientOrderId (str) – A unique id for the order. Automatically generated if not sent.
-icebergQty (decimal) – Used with LIMIT, STOP_LOSS_LIMIT, and TAKE_PROFIT_LIMIT to create an iceberg order.
-newOrderRespType (str) – Set the response JSON. ACK, RESULT, or FULL; default: RESULT.
-recvWindow (int) – the number of milliseconds the request is valid for
```

Mais abaixo, tem exemplos de:

- Response ACK
- Response RESULT
- Response FULL
- Possíveis Exceptions

Se você quer realmente entender a fundo todos os parâmetros (obrigatórios e opcionais) e todos os resultados de saída, visitar o link da API da Binance.

Antes de encerrar, vamos pelo menos cancelar a ordem enviada. 

Exercício: Realizar o cancelamento sozinho utilizando a documentação? Não existe um método para cancelar todas. Analise o resultado de sáida - isso é importante porque um cancelamento pode ser **rejeitado**.

Você perdeu e não anotou o OrderId?
```python
for order in client.get_open_orders():
    print(order['orderId'])
```
---

O PyPI (https://pypi.org/), ou Python Package Index, é um repositório online centralizado para a distribuição de pacotes de software escritos na linguagem de programação Python. Funcionando como um vasto armazém de módulos e bibliotecas, o PyPI permite que desenvolvedores publiquem seus pacotes de software e os disponibilizem para instalação e uso por outros. Usuários podem facilmente instalar pacotes usando ferramentas como pip, o gerenciador de pacotes padrão do Python. Essencial para a comunidade Python, o PyPI facilita o compartilhamento e a colaboração em projetos de software, proporcionando acesso a uma rica biblioteca de recursos que podem ser usados para desenvolver novas aplicações rapidamente e com maior eficiência.

O Read the Docs (https://about.readthedocs.com/) é uma plataforma online amplamente utilizada que permite aos desenvolvedores criar, hospedar e gerenciar documentação de projetos de software de forma gratuita e automatizada. Suportando formatos de documentação como Sphinx e MkDocs, a plataforma facilita a importação de documentos diretamente de repositórios no GitHub, Bitbucket, GitLab, entre outros, mantendo a documentação sempre sincronizada com as atualizações do código-fonte. Além de fornecer ferramentas para criar documentação clara e navegável, o Read the Docs também oferece funcionalidades como versionamento automático e suporte para vários idiomas, tornando-se uma ferramenta essencial para projetos que buscam manter suas documentações acessíveis e atualizadas para desenvolvedores e usuários finais.

Normalmente bibliotecas que são instaladas via **pip** utilizam essa plataforma para hospedar a documentação da biblioteca. Por isso é importante sempre manter uma documentação mínima em formato **docstring** (https://en.wikipedia.org/wiki/Docstring). Dica: ChatGPT e co-pilot são muito bons em montar essa documentação.

In [ ]:
client.create_order(symbol="BTCUSDT",             # Pair to trade
                    side=SIDE_SELL,               # Order side
                    type=ORDER_TYPE_LIMIT,        # Order type (market or limit)
                    quantity=0.1,                 # Quantity (you must have position to sell)
                    price = 130000,                # Price (try to avoid filling the order)
                    timeInForce=TIME_IN_FORCE_GTC # How long the market you last (GTC = good til cancel)
                   )

In [ ]:
for order in client.get_open_orders():
    orderid = order['orderId']
    print(order['orderId'])

In [ ]:
client.cancel_order(symbol="BTCUSDT", orderId=orderid)

# V - Recebendo atualizações das ordens e da posição

Como explicado anteriormente, o evento de preenchimento de ordens e consequentemente mudança de posição da carteira são assíncronos. É possível que uma ordem enviada no ano passado seja preenchido somente hoje. Também é possível que seja parcialmente preenchida em momentos distintos.

Logo, é sabido que precisamos também utilizar um websocket para receber essas atualizações. E uma primeira epifânia seria: "Posso usar um websocket multiplexado (múltiplas conexões) para receber todos os eventos por uma sessão".

Como você é um aluno sagaz e leu a documentação, deve ter lido: "These streams can include the depth, kline, ticker and trade streams **but not the user stream which requires extra authentication**." (https://python-binance.readthedocs.io/en/latest/websockets.html#id1)

Portanto teremos que abrir uma conexão diferente para receber as atualizações de ordens e posição. Copie o **PRIMEIRO** código utilizado para receber os preços, com duas pequenas alterações:

```python
        ...
        # start any sockets here, i.e a trade socket
        # Comentar a linha referente ao kline:
        # ts = bm.kline_socket('BTCUSDT', interval=KLINE_INTERVAL_1MINUTE)
        # Trocar para:
        ts = bm.user_socket()
        # then start receiving messages
        ...
...
# Comente as últimas duas linhas:
# Espere 15 segundos para cancelar a tarefa main()
# await asyncio.sleep(15)
# task.cancel()
```

Agora envie uma ordem e olhe a saida:

```
Evento: {'e': 'executionReport', 'E': 1713302017584, 's': 'BTCUSDT', 'c': 'g8lDmcSDAF2U8webCSm3WI', 'S': 'SELL', 'o': 'LIMIT', 'f': 'GTC', 'q': '0.10000000', 'p': '90000.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'NEW', 'X': 'NEW', 'r': 'NONE', 'i': 1875593, 'l': '0.00000000', 'z': '0.00000000', 'L': '0.00000000', 'n': '0', 'N': None, 'T': 1713302017584, 't': -1, 'I': 4427081, 'w': True, 'm': False, 'M': False, 'O': 1713302017584, 'Z': '0.00000000', 'Y': '0.00000000', 'Q': '0.00000000', 'W': 1713302017584, 'V': 'EXPIRE_MAKER'}
Evento: {'e': 'outboundAccountPosition', 'E': 1713302017585, 'u': 1713302017584, 'B': [{'a': 'BTC', 'f': '0.90000000', 'l': '0.10000000'}, {'a': 'USDT', 'f': '10000.00000000', 'l': '0.00000000'}]}
```

Cancele a ordem, terá um novo evento:

```
Evento: {'e': 'executionReport', 'E': 1713302017584, 's': 'BTCUSDT', 'c': 'g8lDmcSDAF2U8webCSm3WI', 'S': 'SELL', 'o': 'LIMIT', 'f': 'GTC', 'q': '0.10000000', 'p': '90000.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': 'g8lDmcSDAF2U8webCSm3WI', 'x': 'CANCELED', 'X': 'CANCELED', 'r': 'NONE', 'i': 1875593, 'l': '0.00000000', 'z': '0.00000000', 'L': '0.00000000', 'n': '0', 'N': None, 'T': 1713302042331, 't': -1, 'I': 4427374, 'w': False, 'm': False, 'M': False, 'O': 1713302017584, 'Z': '0.00000000', 'Y': '0.00000000', 'Q': '0.00000000', 'W': 1713302017584, 'V': 'EXPIRE_MAKER'}
Evento: {'e': 'outboundAccountPosition', 'E': 1713302042332, 'u': 1713302042331, 'B': [{'a': 'BTC', 'f': '1.00000000', 'l': '0.00000000'}, {'a': 'USDT', 'f': '10000.00000000', 'l': '0.00000000'}]}
```

Vamos analisar as saídas:

- Cada movimentação gera 2 eventos: executionReport e outboundAccountPosition
- **executionReport**: atualiza o status da ordem. Se compararmos os dois reports, um tem o o Status 'x' NEW e o outro CANCELED
- **outboundAccountPosition**: atualiza o saldo atual na carteira, onde 'B' contém uma lista de ativos impactados. 'a' é o asset, 'f' é a quantidade free e 'l' a quantidade locked. Se notar, no primeiro bloco tínhamos 0.9 BTC free e 0.1 locked, justamente por se tratar de uma ordem de venda de 0.1 BTC.

Próximo passo é capturar um preenchimento de ordem.

**Exercício**: envie (via API, óbvio) uma SELL MKT Order para BTCUSDT, venda 0.01 BTC. Capture as saídas do executionReport e outboundAccountPosition. 

Além do NEW e do FILLED no final, você deve ter recebido também diversos:
```
Evento: {'e': 'executionReport', 'E': 1713302801791, 's': 'BTCUSDT', 'c': 'eOKftN6Vyjt1ZnN8ahRz69', 'S': 'SELL', 'o': 'MARKET', 'f': 'GTC', 'q': '0.10000000', 'p': '0.00000000', 'P': '0.00000000', 'F': '0.00000000', 'g': -1, 'C': '', 'x': 'TRADE', 'X': 'FILLED', 'r': 'NONE', 'i': 1878322, 'l': '0.00400000', 'z': '0.10000000', 'L': '62427.37000000', 'n': '0.00000000', 'N': 'USDT', 'T': 1713302801790, 't': 681281, 'I': 4433658, 'w': False, 'm': False, 'M': True, 'O': 1713302801790, 'Z': '6351.82619260', 'Y': '249.70948000', 'Q': '0.00000000', 'W': 1713302801790, 'V': 'EXPIRE_MAKER'}
```

Você consegue interpretar a mensagem? Dica: documentação

---

Por fim você deve ter recebido também uma atualização na posição:

```
Evento: {'e': 'outboundAccountPosition', 'E': 1713302801791, 'u': 1713302801790, 'B': [{'a': 'BTC', 'f': '0.90000000', 'l': '0.00000000'}, {'a': 'USDT', 'f': '16351.82619260', 'l': '0.00000000'}]}
```

Isso significa que você tem agora 0.9 BTC, sem locked, e 16352 dólares em USDT.

Cancele a task:
```python
task.cancel()
```

---

Obviamente, manter as mensagens dessa forma é nada amigável. Em CNTP, esses dados iriam para um banco de dados e seriam exibidos em um Dashboard bem bonito. Então vamos impovisar um mini dashboard com pywidgets aqui mesmo no notebook.

Vamos criar 4 caixas de texto:

```python
import ipywidgets as widgets
from IPython.display import display

btc_free = widgets.Text(value='0', description='BTC Free:', disabled=True);
btc_lock = widgets.Text(value='0', description='BTC Locked:', disabled=True);
usdt_free = widgets.Text(value='0', description='USDT Free:', disabled=True);
usdt_lock = widgets.Text(value='0', description='USDT Locked:', disabled=True);

display(btc_free);
display(btc_lock);
display(usdt_free);
display(usdt_lock);

# Dicionário de mapeamento entre os nomes e as caixas de texto
textboxes = {'BTC_f': btc_free, 'BTC_l': btc_lock, 'USDT_f': usdt_free, 'USDT_l': usdt_lock}
```

Antes de rodar esse código, instale a biblioteca ipywidgets

**Exercício**: Complementar o código acima para preencher a posição inicial dos ativos. Ver o capítulo II. Dica: Você pode trocar o valor de uma caixa de texto usando: textboxes['BTC_f'].value = 10.

Agora precisamos amarrar a mensagem recebida para escrever nas caixas de texto. Vamos fazer algo similar ao feito no capítulo III:

```python
async def process_user(message):
    if message['e'] == 'outboundAccountPosition':
        for asset in message['B']:
            name = asset['a']
            textboxes[f'{name}_f'].value = asset['f']
            textboxes[f'{name}_l'].value = asset['l']
            
```

**Exercício**: Integrar esse código para preencher automaticamente os valores a cada execução. Ver o capítulo III.

- Mande uma ordem LMT e veja se os valores nas caixas de texto mudaram. Cancele a ordem.

Repita o passo anterior até funcionar.

---

Certifique-se de que você cancelou todas as ordens.

Para finalizar cancele a task novamente para fechar a conexão (que havia ficado aberta):

```python
task.cancel()
```

Agora vamos integrar tudo em um programa só e adicionar uma estratégia.

In [ ]:
async def process_user(message):
    if message['e'] == 'outboundAccountPosition':
        for asset in message['B']:
            name = asset['a']
            textboxes[f'{name}_f'].value = asset['f']
            textboxes[f'{name}_l'].value = asset['l']

    # Coloque aqui o código para tratar outros eventos
            
async def main(api_key, api_secret):
    print('Connection started!')
    client = None
    try:
        client = await AsyncClient.create(api_key=api_key, api_secret=api_secret, testnet=True)
        bm = BinanceSocketManager(client)
        # start any sockets here, i.e a trade socket
        ts = bm.user_socket()
        # then start receiving messages
        async with ts as tscm:
            while True:
                res = await tscm.recv()
                await process_user(res)
                
    except asyncio.CancelledError:
        print('Task was cancelled!')
    finally:
        if client:
            await client.close_connection()
        print('Connection finished!')    

loop = asyncio.get_event_loop()
task = loop.create_task(main(api_key, api_secret))

# Espere 15 segundos para cancelar a tarefa main()
#await asyncio.sleep(15)
#task.cancel()

In [ ]:
client.create_order(symbol="BTCUSDT",             # Pair to trade
                    side=SIDE_SELL,               # Order side
                    type=ORDER_TYPE_MARKET,       # Order type (market or limit)
                    quantity=0.1,                 # Quantity (you must have position to sell)
                    
                    
                   )

In [ ]:
task.cancel()

In [ ]:
import ipywidgets as widgets
from IPython.display import display

btc_free = widgets.Text(value='0', description='BTC Free:', disabled=True);
btc_lock = widgets.Text(value='0', description='BTC Locked:', disabled=True);
usdt_free = widgets.Text(value='0', description='USDT Free:', disabled=True);
usdt_lock = widgets.Text(value='0', description='USDT Locked:', disabled=True);

display(btc_free);
display(btc_lock);
display(usdt_free);
display(usdt_lock);

# Dicionário de mapeamento entre os nomes e as caixas de texto
textboxes = {'BTC_f': btc_free, 'BTC_l': btc_lock, 'USDT_f': usdt_free, 'USDT_l': usdt_lock}

position = client.get_account()
for balance in position['balances']:
    if balance['asset'] in ['BTC', 'USDT']:
        name = balance['asset']
        textboxes[f'{name}_f'].value = balance['free']
        textboxes[f'{name}_l'].value = balance['locked']
